In [1]:
import pandas as pd
import pickle
import re
import os
from collections import defaultdict
from time import time
import csv
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
import multiprocessing
from gensim.models import Word2Vec
import numpy as np
import logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
#importing list of slang words

In [4]:
slang_dict = {}

for _row in csv.reader(open("slang.txt", "r"), delimiter = "="):
    if _row != "":
        slang_dict[_row[0]] = _row[1]

In [5]:
#defining a list of punctuation symbols

In [6]:
punctuation_list = [",", ".", "!", "?", "$", "%", "^", "&", "*", ")", "(", "=", \
                    ":", "|", ";", "<", ">", "~", "‘", "'", "/", "\\", "’", "-"]

In [7]:
# defining a class that goes through all csv files(1 file per day) within a folder, applies all 
#the cleaning functionswithin it and saves the cleaned tweets in a dataframe, saves each day's dataframe
#in a pickled file in a folder title "cleaned_folder" 

In [8]:
class tweet_cleaner:
    
    def __init__(self, _file, _folder, save_folder, necessary_columns = ["id", "conversation_id", "date", "time", \
                                                            "user_id", "tweet", "retweets_count",\
                                                            "replies_count", "likes_count", "retweet"]):
        self.folder = _folder
        self.file = _file
        self.save_folder = save_folder
        self.necessary_columns = necessary_columns
        self.df = pd.read_csv(os.path.join(os.path.abspath(self.folder), self.file))
        self.cleaned_df = self.clean_tweet()
        self.output = self.save_output()
        
        
    
    @staticmethod
    #find the share of capital letters used in a tweet
    def find_caps_share(tweet):
        caps_length = len(re.findall(r"[A-Z]", tweet))
        tweet_length = len(re.sub(r"\s", "", tweet))
        
        return caps_length/(1 + tweet_length)
    
    @staticmethod
    #replace contractions such as "can't", "won't" etc with "cannot", "will not"
    def remove_contractions(tweet):
        tweet = tweet.lower()
        tweet = re.sub(r"can\s?'\s?t", "cannot", tweet)
        tweet = re.sub(r"won\s?'\s?t", "will not", tweet)
        tweet = re.sub(r"n\s?'\s?t", " not", tweet)
        tweet = re.sub(r"'\s?ve", " have", tweet)
        tweet = re.sub(r"'\s?ll", " will", tweet)
        tweet = re.sub(r"'\s?re", " are", tweet)
        tweet = re.sub(r"'\s?m", " am", tweet)
        tweet = re.sub(r"'\s?d", " would", tweet)
        
        tweet = re.sub(r"can\s?‘\s?t", "cannot", tweet)
        tweet = re.sub(r"won\s?‘\s?t", "will not", tweet)
        tweet = re.sub(r"n\s?‘\s?t", " not", tweet)
        tweet = re.sub(r"‘\s?ve", " have", tweet)
        tweet = re.sub(r"‘\s?ll", " will", tweet)
        tweet = re.sub(r"‘\s?re", " are", tweet)
        tweet = re.sub(r"‘\s?m", " am", tweet)
        tweet = re.sub(r"‘\s?d", " would", tweet)
        
        tweet = re.sub(r"can\s?’\s?t", "cannot", tweet)
        tweet = re.sub(r"won\s?’\s?t", "will not", tweet)
        tweet = re.sub(r"n\s?’\s?t", " not", tweet)
        tweet = re.sub(r"’\s?ve", " have", tweet)
        tweet = re.sub(r"’\s?ll", " will", tweet)
        tweet = re.sub(r"’\s?re", " are", tweet)
        tweet = re.sub(r"’\s?m", " am", tweet)
        tweet = re.sub(r"’\s?d", " would", tweet)
        
        return " ".join(tweet.split())
    
    @staticmethod
    #remove all the punctuation symbols defined before from the tweet
    def remove_punctuation(tweet):
        for _punct in punctuation_list:
            tweet = tweet.replace(_punct, " ")
        return " ".join(tweet.split())
    
    @staticmethod
    #return the hashtag used in the tweet
    def tweet_hashtag(tweet):
        splits = tweet.split()
        splits = [split[1:] for split in splits if split[0] == "#"]
        return " ".join(splits)
    
    @staticmethod
    #return all users mentioned in the tweet
    def tweet_target(tweet):
        splits = tweet.split()
        splits = [split[1:] for split in splits if split[0] == "@"]
        return " ".join(splits)
    
    @staticmethod
    #remove any users mentioned in the tweet
    def remove_targets(tweet):
        tweet = re.sub(r"@_*\w+(_+\w+)*_*", " ", tweet)
        return " ".join(tweet.split())
    
    @staticmethod
    #remove any hashtags in the tweet
    def remove_hashtags(tweet):
        tweet = re.sub(r"#\w+", " ", tweet)
        return " ".join(tweet.split())
    
    @staticmethod
    #remove all urls in the tweet
    def remove_url(tweet):
        tweet = re.sub(r"https?://t.co/\w+", " ", tweet)
        return " ".join(tweet.split())
    
    @staticmethod
    #identify all emojis used in the tweet, find their meaning in string format, return a string made up of all emoji meanings
    def find_emojis(tweet):
        emojis = []
        for _emoji in UNICODE_EMOJI:
            if _emoji in tweet:
                for i in range(len((re.findall(_emoji, tweet)))):
                    emojis.append(UNICODE_EMOJI[_emoji].replace(":", ""))
            else:
                pass
        
        return " ".join(emojis)
    
    @staticmethod
    #remove all emojis in the tweet
    def remove_emojis(tweet):
        for _emoji in UNICODE_EMOJI:
            tweet = tweet.replace(_emoji, " ")
        return " ".join(tweet.split())
    
    @staticmethod
    #replace all slang words in the tweet by their meaning using previously imported list
    def remove_slang(tweet):
        new_text = []
        for _word in tweet.split():
            if _word.upper() in slang_dict.keys():
                new_text.append(slang_dict[_word.upper()])
            else:
                new_text.append(_word)
        
        return " ".join(new_text)
    
    @staticmethod
    #remove remaning punctuation marks
    def make_alphabetic(tweet):
        tweet = tweet.replace("…", " ")
        tweet = tweet.replace("_", " ")
        tweet = tweet.replace("@", " ")
        tweet = tweet.replace("#", " ")
        return " ".join([_word for _word in tweet.split() if _word.isalpha() and len(_word) > 1 \
                        and _word != "rt"])
    
    #create dataframe out of day's data. create the new columns "cleaned_tweet", "caps_share", "target", "hashtag", 
    #"emojis" 
    def clean_tweet(self):
        _cleaned = self.df.copy()
        _cleaned = _cleaned.where(_cleaned.language == "en").dropna(subset = ["language"])
        _cleaned.reset_index(drop = True, inplace = True)
        _cleaned = _cleaned[self.necessary_columns]
        _cleaned["caps_share"] = _cleaned["tweet"].apply(self.find_caps_share)
        _cleaned["cleaned_tweet"] = _cleaned["tweet"].apply(self.remove_contractions)
        _cleaned["cleaned_tweet"] = _cleaned["cleaned_tweet"].apply(self.remove_url)
        _cleaned["cleaned_tweet"] = _cleaned["cleaned_tweet"].apply(self.remove_punctuation)
        _cleaned["target"] = _cleaned["cleaned_tweet"].apply(self.tweet_target)
        _cleaned["hashtag"] = _cleaned["cleaned_tweet"].apply(self.tweet_hashtag)
        _cleaned["cleaned_tweet"] = _cleaned["cleaned_tweet"].apply(self.remove_targets)
        _cleaned["cleaned_tweet"] = _cleaned["cleaned_tweet"].apply(self.remove_hashtags)
        _cleaned["emojis"] = _cleaned["cleaned_tweet"].apply(self.find_emojis)
        _cleaned["cleaned_tweet"] = _cleaned["cleaned_tweet"].apply(self.remove_emojis)
        _cleaned["cleaned_tweet"] = _cleaned["cleaned_tweet"].apply(self.remove_slang)
        _cleaned["cleaned_tweet"] = _cleaned["cleaned_tweet"].apply(self.make_alphabetic)
        _cleaned.dropna(subset = ["cleaned_tweet"], inplace = True)
        _cleaned.reset_index(drop = True, inplace = True)
        return _cleaned
    
    #save the dataframe as pickled file titled "cleaned filename" in a new folder titled "cleaned foldername" with 
    def save_output(self):
        try:
            state_name = self.folder.rpartition("\\")[-1]
            dirname = "cleaned {}".format(state_name)
            folder_name = self.save_folder + "\\" +  dirname
            os.mkdir(folder_name)
            print("Folder {} created".format(dirname))
        except FileExistsError:
            pass
        with open(os.path.join(os.path.abspath(folder_name), "cleaned {}.pkl".format(self.file)), "wb") as f:
            pickle.dump(self.cleaned_df, f)

In [9]:
#running a loop through all states in the Raw Data folder and saving the cleaned output in the 
#Cleaned Data folder

In [ ]:
for _folder in os.listdir("Raw Data"):
    t_1 = time()
    file_list = [_file for _file in os.listdir("raw data\\{}".format(_folder)) if ".csv" in _file]
    for _file in file_list:
        tweet_cleaner(_file, os.path.join("raw data", _folder), "Cleaned Data")
    print("{} completed".format(_folder))
    print("time taken: {} mins".format(round((time() - t_1)/60, 2)))
print("time taken: {} mins".format(round((time() - t)/60, 2)))

In [10]:
#collecting all tweets written in Roman script to train word2vec

In [11]:
class tweet_collecter:
    
    def __init__(self, folder):
        self.folder = folder
        self.file_list = [file for file in os.listdir(self.folder) if ".csv" in file]
        self.tweets = self.extract_tweets()
     
    @staticmethod
    def push_to_csv(_df):
        _df.to_csv("word2vec_tweets.csv", mode = "a", header = False, index = False)
        with open("word2vec_tweets.csv") as f:
            f.close()
        
        
    def extract_tweets(self):
        for file in self.file_list:
            df = pickle.load(open(os.path.join(os.path.abspath(self.folder), file), "rb"))
            df = df.dropna(subset = ["cleaned_tweet"])
            df.drop_duplicates(subset = ["cleaned_tweet"], inplace = True)
            df.reset_index(inplace = True, drop = True)
            self.push_to_csv(df.loc[:, ["cleaned_tweet", "id"]])

In [ ]:
t_1 = time()
for _folder in os.listdir("Cleaned Data"):
    t_2 = time()
    _path = os.path.join("Cleaned Data", _folder)
    tweet_collecter(_path)
    print("completed {}".format(_folder))
    print("time taken to complete: {} mins".format(round((time() - t_2)/60, 2)))
print("time taken to complete program: {} mins".format(round((time() - t_1)/60, 2)))

In [12]:
#reading the file of collected tweets and removing duplicate tweets from them

In [ ]:
tweets = pd.read_csv("word2vec_tweets.csv", header = None)

In [ ]:
tweets.drop_duplicates(subset = [0], inplace = True)
tweets.drop_duplicates(subset = [1], inplace = True)

In [13]:
#writing the tweets to a txt file for feeding into the word2vec 

In [ ]:
with open("word2vec_tweets.txt", "w", encoding = "utf-8") as f:
    for _row in tweets[0]:
        f.write(str(_row))
        f.write("\n")

In [15]:
#saving tweet ids to a pickle file

In [ ]:
pickle.dump(tweets[1].values, open("unique tweet ids.pkl", "wb"))

In [16]:
#finding tweet ids of all tweets across all states, taking care to drop all duplicates within the same day and
#then adding all these ids to a list. Only the unique values in this list are retained as the final ids of
#state

In [ ]:
t = time()
state_list = [state for state in os.listdir("Cleaned Data")]
for state in state_list:
    ids = np.array(0)
    t_1 = time()
    _path = os.path.join("Cleaned Data", state)
    file_list = [file for file in os.listdir(_path) if ".csv" in file]
    for file in file_list:
        df = pickle.load(open(os.path.join(_path, file), "rb"))
        df = df.where(df["id"].isin(tweets[1].values)).dropna(subset = ["id"])
        df.drop_duplicates(subset = ["id"], inplace = True)
        df.drop_duplicates(subset = ["cleaned_tweet"], inplace = True)
        ids = np.append(ids, df["id"].values)
    ids = np.unique(ids)
    pickle.dump(ids, open(os.path.join(_path, "{} ids.pkl".format(state.split()[-1])), "wb"))
print("time taken: {} mins".format(round((time() - t)/60, 2)))

In [ ]:
#eliminating duplicate tweets across states. code snippet compares the ids list of two states, keeping only the
#unique values of list 2 in list 2, and saving list 1 as is. b

In [ ]:
state_list = os.listdir("Cleaned Data")
for state in state_list:
    search_list = state_list[state_list.index(state) + 1: ]
    _path = os.path.join("Cleaned Data", state)
    state_ids = pickle.load(open(os.path.join(_path, "{} ids.pkl".format(state.split()[-1])), "rb"))
    for _state in search_list:
        _path_2 = os.path.join("Cleaned Data", _state)
        _state_ids = pickle.load(open(os.path.join(_path_2, "{} ids.pkl".format(_state.split()[-1])), "rb"))
        mask = np.isin(_state_ids, state_ids, invert = True)
        _state = _state_ids[mask]
        pickle.dump(_state_ids, open(os.path.join(_path_2, "{} ids.pkl".format(_state.split()[-1])), "wb"))
    pickle.dump(state_ids, open(os.path.join(_path, "{} ids.pkl".format(state.split()[-1])), "wb"))

In [ ]:
#training the word2vec model